In [2]:
import numpy as np
import pandas as pd
import sklearn.preprocessing
import xgboost
print("xgboost", xgboost.__version__)

xgboost 0.7


In [57]:
train_df = pd.read_csv("/Users/BrianLee/Desktop/input/titanic_train.csv")
test_df = pd.read_csv("/Users/BrianLee/Desktop/input/titanic_test.csv")
print(train_df.shape, test_df.shape)

(891, 12) (418, 11)


In [15]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [58]:
y_train = train_df['Survived']
train_df = train_df.drop('Survived', axis = 1)

In [59]:
train_df = train_df.drop('Name', axis = 1)
test_df = test_df.drop('Name', axis = 1)

In [60]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le_count = 0
traintest = pd.concat([train_df, test_df])

for col in traintest:
    if traintest[col].dtype == 'object':
        if (len(list(traintest[col].unique()))) <= 2:
            le.fit(traintest[col])
            traintest[col] = le.transform(traintest[col])
            le_count += 1
print('%d columns label encoded. ' % le_count)

1 columns label encoded. 


In [61]:
traintest = pd.get_dummies(traintest)
print(traintest.shape)

(1309, 1125)


In [62]:
train = traintest[:891]
test = traintest[891:]

#align data
train, test = train.align(test, join = 'inner', axis = 1)

print(train.shape, test.shape)

(891, 1125) (418, 1125)


In [63]:
from sklearn.preprocessing import MinMaxScaler, Imputer

x_train = train.copy()
x_test = test.copy()

imputer = Imputer(strategy = 'median')
scaler = MinMaxScaler(feature_range = (0,1))

imputer.fit(x_train)
x_train = imputer.transform(x_train)
x_test = imputer.transform(x_test)

scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [64]:
#fitting with basic xgboost
import xgboost as xgb

#dtrain = xgb.DMatrix('/Users/BrianLee/Desktop/input/titanic_train.csv?format=csv&label_column=1')
#dtest = xgb.DMatrix('/Users/BrianLee/Desktop/input/titanic_test.csv?format=csv&label_column=1')
dtrain = xgb.DMatrix(x_train, label = y_train)
dtest = xgb.DMatrix(x_test)


In [70]:
#Setting up parameters

param = {'max_depth': 2000, 'eta': 0.001, 'silent': 1, 'objective': 'binary:logistic'}
param['nthread'] = 4
param['eval_metric'] = 'auc'
param['eval_metric'] = ['auc', 'ams@0']
evallist = [(dtest, 'eval'), (dtrain, 'train')]

In [71]:
#plst = param.items()

In [72]:
#Training

num_round = 1000
bst = xgb.train(param, dtrain, num_round)

In [73]:
#Predictions
results = bst.predict(dtest)
results = [int(round(value)) for value in results]

In [74]:
results = pd.Series(results, name = 'Survived')
submission = pd.concat([pd.Series(test_df['PassengerId'], name = 'PassengerId' ), results], axis = 1)
submission.to_csv("titanic_xgb_submission.csv",index = False)